In [4]:
from azure.ai.ml import command, Input, Output
from azure.ai.ml.dsl import pipeline
from azure.ai.ml.constants import AssetTypes, InputOutputModes
from azure.identity import DefaultAzureCredential
from azure.ai.ml import MLClient

In [3]:
sub_id = "b746917e-ceb7-4ae0-81e6-3ccd893cb0de"
rg = "dpv2"
workspace = "dpv2-wks"

ml_client = MLClient(DefaultAzureCredential(), sub_id, rg, workspace)

In [5]:
from azure.ai.ml import load_component
from azure.ai.ml.entities import Environment

In [6]:
spark_kmeans = load_component("./components/spark_kmeans.yml")

In [8]:
show_output = command(
    inputs=dict(
        spark_output=Input(
            type=AssetTypes.URI_FOLDER
        )
    ),
    command = "ls ${{inputs.spark_output}}",
    environment="AzureML-sklearn-0.24-ubuntu18.04-py37-cpu:1",
)

In [10]:
from msilib.schema import Directory


@pipeline()
def spark_pipeline(train_data):
    kmeans_clustering_step = spark_kmeans(file_input = train_data)
    kmeans_clustering_step.compute = "spark-cluster"
    kmeans_clustering_step.outputs.output.mode = InputOutputModes.DIRECT

    show_output_step = show_output(spark_output = kmeans_clustering_step.outputs.output)

sample_data = Input(
    path="./data/sample_kmeans_data.txt",
    type=AssetTypes.URI_FILE,
    mode=InputOutputModes.DIRECT
)

job = spark_pipeline(train_data = sample_data)

job.settings.default_compute  = "cpu-cluster"

ret_job = ml_client.jobs.create_or_update(job, experiment_name="spark_demo")

Uploading components (0.0 MBs): 100%|##########| 2166/2166 [00:01<00:00, 2009.89it/s]


Uploading sample_kmeans_data.txt (< 1 MB): 100%|##########| 124/124 [00:00<00:00, 459B/s]




In [11]:
ml_client.jobs.stream(ret_job.name)

RunId: silver_stick_5wjg26tn4f
Web View: https://ml.azure.com/runs/silver_stick_5wjg26tn4f?wsid=/subscriptions/b746917e-ceb7-4ae0-81e6-3ccd893cb0de/resourcegroups/dpv2/workspaces/dpv2-wks

Streaming logs/azureml/executionlogs.txt

[2022-10-06 06:34:16Z] Submitting 1 runs, first five are: e5be719d:802ee6f9-cff3-4be5-8750-4553b9be703c
[2022-10-06 06:41:55Z] Execution of experiment failed, update experiment status and cancel running nodes.

Execution Summary
RunId: silver_stick_5wjg26tn4f
Web View: https://ml.azure.com/runs/silver_stick_5wjg26tn4f?wsid=/subscriptions/b746917e-ceb7-4ae0-81e6-3ccd893cb0de/resourcegroups/dpv2/workspaces/dpv2-wks


JobException: Exception : 
 {
    "error": {
        "code": "UserError",
        "message": "Pipeline has some failed steps. See child run or execution logs for more details.",
        "message_format": "Pipeline has some failed steps. {0}",
        "message_parameters": {},
        "reference_code": "PipelineHasStepJobFailed",
        "details": []
    },
    "environment": "eastus",
    "location": "eastus",
    "time": "2022-10-06T06:41:56.060864Z",
    "component_name": ""
} 